In [ ]:
! pip install -U bitsandbytes pysftp Faker -q

In [ ]:
import json

In [ ]:
cluster_numeclature_path = "/content/drive/MyDrive/data/CSM/cluster_numeculature.json"
session_nomic_embeddings_path = (
    "/content/drive/MyDrive/data/CSM/embeddings_test_nomic.json"
)
session_mistral_embeddings_path = (
    "/content/drive/MyDrive/data/CSM/embeddings_test_mistral.json"
)
old_session_mistral_embeddings_path = (
    "/content/drive/MyDrive/data/CSM/session_embeddings_Mistral-7B-v0.1.json"
)
session_mistral_embeddings_path = (
    "/content/drive/MyDrive/data/CSM/badge/embeddings_badges_mistral.json"
)

In [ ]:
with open(cluster_numeclature_path, "r") as f:
    numeculature_dict = json.load(f)

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import json
from tqdm import tqdm
import os
import gc

In [ ]:
# del model
# del tokenizer
torch.cuda.empty_cache()
gc.collect()

# Mistral 7B

In [ ]:
# model_id = "mistralai/Mixtral-8x7B-v0.1"
model_id = "mistralai/Mistral-7B-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id, device_map="auto", load_in_8bit=True)


cluster_numeculature_embeddings = {}
for key, value in tqdm(numeculature_dict.items()):
    # Tokenize the input text and convert to tensor
    inputs = tokenizer(value, return_tensors="pt", truncation=True, max_length=512)
    inputs = inputs.to("cuda")

    # Forward pass, get all hidden states
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)

    # Extract hidden states of the second-to-last layer
    hidden_states = outputs.hidden_states
    second_to_last_layer = hidden_states[-2]

    cluster_numeculature_embeddings[key] = (
        second_to_last_layer.mean(dim=1).cpu().numpy()
    )

if os.path.exists("/content/drive/MyDrive/data/CSM/embeddings") != True:
    os.makedirs("/content/drive/MyDrive/data/CSM/embeddings")

# Save embeddings to a file
with open(
    f'/content/drive/MyDrive/data/CSM/embeddings/cluster_numeculature_embeddings_{model_id.split("/")[1]}.json',
    "w",
    encoding="utf-8",
) as f:
    json.dump(
        {k: v.tolist() for k, v in cluster_numeculature_embeddings.items()},
        f,
        ensure_ascii=False,
        indent=4,
    )

In [ ]:
import numpy as np
import json

with open(
    "/content/drive/MyDrive/data/CSM/embeddings/cluster_numeculature_embeddings_Mistral-7B-v0.1.json"
) as json_file:
    cluster_numeculature_dict_mistral = json.load(json_file)

with open(
    "/content/drive/MyDrive/data/CSM/badge/embeddings_badges_mistral.json"
) as json_file:
    session_embeddings_Mistral_7B_dict = json.load(json_file)

In [ ]:
len(session_embeddings_Mistral_7B_dict.keys())

In [ ]:
len(session_embeddings_Mistral_7B_dict["BadgeId_22A5D99"][0])

In [ ]:
cluster_numeculature_dict_mistral.keys()

In [ ]:
len(cluster_numeculature_dict_mistral["Networking"][0])

In [ ]:
np.array(session_embeddings_Mistral_7B_dict["BadgeId_22A5D99"]).shape

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(
    np.array(session_embeddings_Mistral_7B_dict["BadgeId_22A5D99"]),
    np.array(cluster_numeculature_dict_mistral["Networking"]),
)[0][0]

In [ ]:
def cosine_distance(A, B):
    dot_product = np.dot(A, B)
    magnitude_A = np.linalg.norm(A)
    magnitude_B = np.linalg.norm(B)
    return 1 - dot_product / (magnitude_A * magnitude_B)


def find_key_of_lowest_value(input_dict):
    # Initialize minimum value and corresponding key
    min_value = float("inf")
    min_key = None

    # Iterate through the dictionary
    for key, value in input_dict.items():
        # Update minimum value and key if a lower value is found
        if value < min_value:
            min_value = value
            min_key = key

    return min_key


def find_key_of_higher_value(input_dict):
    # Initialize minimum value and corresponding key
    min_value = float("-inf")
    min_key = None

    # Iterate through the dictionary
    for key, value in input_dict.items():
        # Update minimum value and key if a lower value is found
        if value > min_value:
            min_value = value
            min_key = key

    return min_key


def sort_dict_by_values(input_dict):
    # Sort the dictionary by its values in ascending order
    sorted_items = sorted(input_dict.items(), key=lambda x: x[1])
    return sorted_items

In [ ]:
clusters = {}
for badge_id, session_embeddings in session_embeddings_Mistral_7B_dict.items():
    cosine_distances = {}
    cosines_similarities = {}
    # print(badge_id)
    for (
        cluster_group_name,
        cluster_numeculature,
    ) in cluster_numeculature_dict_mistral.items():
        cosine_distances[cluster_group_name] = cosine_distance(
            np.squeeze(session_embeddings), np.squeeze(cluster_numeculature)
        )
        cosines_similarities[cluster_group_name] = cosine_similarity(
            np.array(session_embeddings), np.array(cluster_numeculature)
        )[0][0]

    # print(json.dumps(cosine_distances, indent=2))
    closest_cluster_group_name = find_key_of_lowest_value(cosine_distances)
    closest_cluster_group_name_cos = find_key_of_higher_value(cosines_similarities)
    if badge_id not in clusters.keys():
        clusters[badge_id] = closest_cluster_group_name_cos
    # print(f"Badge {badge_id}, closer distance {closest_cluster_group_name}, closer cosine {closest_cluster_group_name_cos}")
    # Sort the dictionary and print the result
    # sorted_cosine_distances = sort_dict_by_values(cosine_distances)
    # print(">>>>>>")
    # for key, value in sorted_cosine_distances:
    #     print(f"{key}: {value}")
    # print(f"    closest cluster: {closest_cluster_group_name}")
    # print("<<<<<<")
    # input()

In [ ]:
len(clusters.keys())

In [ ]:
for key in list(clusters.keys())[0:5]:
    print(key, clusters[key])

# LLama


In [ ]:
del model

del tokenizer
torch.cuda.empty_cache()
gc.collect()

In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
model_id = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id, device_map="auto", load_in_8bit=True)

In [ ]:
output_path = "/content/drive/MyDrive/data/CSM/badge/embeddings_badges_llama.json"

In [ ]:
cluster_numeculature_embeddings_llama = {}
for key, value in tqdm(numeculature_dict.items()):
    # Tokenize the input text and convert to tensor
    inputs = tokenizer(value, return_tensors="pt", truncation=True, max_length=512)
    inputs = inputs.to("cuda")

    # Forward pass, get all hidden states
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)

    # Extract hidden states of the second-to-last layer
    hidden_states = outputs.hidden_states
    second_to_last_layer = hidden_states[-2]

    cluster_numeculature_embeddings_llama[key] = (
        second_to_last_layer.mean(dim=1).cpu().numpy()
    )

if os.path.exists("/content/drive/MyDrive/data/CSM/embeddings") != True:
    os.makedirs("/content/drive/MyDrive/data/CSM/embeddings")

# Save embeddings to a file
with open(
    f'/content/drive/MyDrive/data/CSM/embeddings/cluster_numeculature_embeddings_{model_id.split("/")[1]}.json',
    "w",
    encoding="utf-8",
) as f:
    json.dump(
        {k: v.tolist() for k, v in cluster_numeculature_embeddings_llama.items()},
        f,
        ensure_ascii=False,
        indent=4,
    )

In [ ]:
f'/content/drive/MyDrive/data/CSM/embeddings/cluster_numeculature_embeddings_{model_id.split("/")[1]}.json'

In [ ]:
with open(
    "/content/drive/MyDrive/data/CSM/embeddings/cluster_numeculature_embeddings_Llama-3.2-3B-Instruct.json"
) as json_file:
    cluster_numeculature_dict_llama = json.load(json_file)

with open(
    "/content/drive/MyDrive/data/CSM/badge/embeddings_badges_llama.json"
) as json_file:
    session_embeddings_LLama = json.load(json_file)

In [ ]:
clusters3 = {}
for badge_id, session_embeddings in session_embeddings_LLama.items():
    cosine_distances = {}
    cosines_similarities = {}
    # print(badge_id)
    for (
        cluster_group_name,
        cluster_numeculature,
    ) in cluster_numeculature_dict_llama.items():
        # cosine_distances[cluster_group_name] = cosine_distance(np.squeeze(session_embeddings), np.squeeze(cluster_numeculature))
        cosines_similarities[cluster_group_name] = cosine_similarity(
            np.array(session_embeddings), np.array(cluster_numeculature)
        )[0][0]

    # print(json.dumps(cosine_distances, indent=2))
    # closest_cluster_group_name = find_key_of_lowest_value(cosine_distances)
    closest_cluster_group_name_cos = find_key_of_higher_value(cosines_similarities)
    if badge_id not in clusters3.keys():
        clusters3[badge_id] = closest_cluster_group_name_cos
    # Sort the dictionary and print the result
    # sorted_cosine_distances = sort_dict_by_values(cosine_distances)
    # print(">>>>>>")
    # for key, value in sorted_cosine_distances:
    #     print(f"{key}: {value}")
    # print(f"    closest cluster: {closest_cluster_group_name}")
    # print("<<<<<<")
    # input()

In [ ]:
len(clusters3.keys())

In [ ]:
for key in list(clusters3.keys())[0:5]:
    print(key, clusters3[key])

# Nomic

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code=True)

In [ ]:
output_path = "/content/drive/MyDrive/data/CSM/badge/embeddings_badges_nomic.json"

In [ ]:
with open(output_path) as json_file:
    session_embeddings_nomic_dict = json.load(json_file)

In [ ]:
len(session_embeddings_nomic_dict["BadgeId_22A5D99"])

In [ ]:
numeculature_dict

In [ ]:
cluster_numeculature_embeddings_nomic = {}
for key, value in tqdm(numeculature_dict.items()):
    # Tokenize the input text and convert to tensor

    cluster_numeculature_embeddings_nomic[key] = model.encode(
        f"search_document: {value}"
    )

In [ ]:
cluster_numeculature_embeddings_nomic.keys()

In [ ]:
len(cluster_numeculature_embeddings_nomic["Networking"])

In [ ]:
np.squeeze(cluster_numeculature_embeddings_nomic["Networking"]).shape

In [ ]:
cosine_similarity(
    np.array(session_embeddings_nomic_dict["BadgeId_22A5D99"]).reshape(1, -1),
    np.array(cluster_numeculature_embeddings_nomic["Networking"]).reshape(1, -1),
)[0][0]

In [ ]:
clusters2 = {}
for badge_id, badge_embeddings in session_embeddings_nomic_dict.items():
    cosine_distances = {}
    cosines_similarities = {}
    # print(badge_id)
    for (
        cluster_group_name,
        cluster_numeculature,
    ) in cluster_numeculature_embeddings_nomic.items():
        # cosine_distances[cluster_group_name] = cosine_distance(np.squeeze(session_embeddings), np.squeeze(cluster_numeculature))
        cosines_similarities[cluster_group_name] = cosine_similarity(
            np.array(badge_embeddings).reshape(1, -1),
            np.array(cluster_numeculature).reshape(1, -1),
        )[0][0]

    # print(json.dumps(cosine_distances, indent=2))
    # closest_cluster_group_name = find_key_of_lowest_value(cosine_distances)
    closest_cluster_group_name_cos = find_key_of_higher_value(cosines_similarities)
    if badge_id not in clusters2.keys():
        clusters2[badge_id] = closest_cluster_group_name_cos

    # Sort the dictionary and print the result
    # sorted_cosine_distances = sort_dict_by_values(cosine_distances)
    # print(">>>>>>")
    # for key, value in sorted_cosine_distances:
    #     print(f"{key}: {value}")
    # print(f"    closest cluster: {closest_cluster_group_name}")
    # print("<<<<<<")
    # input()

In [ ]:
len(clusters2.values())

In [ ]:
for key in list(clusters2.keys())[0:5]:
    print(key, clusters2[key])

In [ ]:
labels = list(clusters2.values())

In [ ]:
from collections import Counter

Counter(labels)

In [ ]:
labels_mistral = list(clusters.values())
Counter(labels_mistral)

In [ ]:
labels_llama = list(clusters3.values())
Counter(labels_llama)

In [ ]:
# clusters3